# Create Table

### Tool Functions

In [50]:
def date(x):
    if type(x) == float:
        return 'null'
    else:
        return(pd.to_datetime(list(x.values())[0], unit='ms'))

def date1(x):
    if type(x) == float:
        return 'null'
    else:
        return(x)

def explode(df, col):
    df[col] = df[col].apply(lambda x: [x] if not isinstance(x, list) else x)
    return df.drop(col, axis=1).join(pd.DataFrame(list(df[col])).stack().reset_index(level=1, drop=True).rename(col)) 

def explode1(x, key):
    if type(x) == dict and key in x.keys(): return x[key]

def checkKey(dic, key):
    if key in dic.keys():
        return True
    else:
        return False

In [16]:
import json    
import pandas as pd
import datetime

receipt_data = pd.read_json("receipts.json", lines=True)
receipt_data['_id'] = receipt_data['_id'].apply(lambda x: list(x.values())[0])
receipt_data['createDate'] = receipt_data['createDate'].apply(lambda x: date(x))
receipt_data['dateScanned'] = receipt_data['dateScanned'].apply(lambda x: date(x))
receipt_data['finishedDate'] = receipt_data['finishedDate'].apply(lambda x: date(x))
receipt_data['modifyDate'] = receipt_data['finishedDate'].apply(lambda x: date1(x))
receipt_data['pointsAwardedDate'] = receipt_data['finishedDate'].apply(lambda x: date1(x))
receipt_data['purchaseDate'] = receipt_data['finishedDate'].apply(lambda x: date1(x))

receipt_data = receipt_data.rename(columns={'_id':'Id'})
receipt_tb = pd.DataFrame(receipt_data, columns=["Id", "bonusPointsEarned", "bonusPointsEarnedReason", "createDate", "dateScanned", "finishedDate", "modifyDate", "pointsAwardedDate", "pointsEarned", "purchaseDate", "purchasedItemCount", "rewardsReceiptStatus", "totalSpent", "userId"])
receipt_tb

,Id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 15:25:31.000,2021-01-03 15:25:31.000,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,500.0,2021-01-03 15:25:31,5.0,FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43.000,2021-01-03 15:24:43.000,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,150.0,2021-01-03 15:24:43,2.0,FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 15:25:37.000,2021-01-03 15:25:37.000,null,null,null,5.0,null,1.0,REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03 15:25:34.000,2021-01-03 15:25:34.000,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,5.0,2021-01-03 15:25:34,4.0,FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,2021-01-03 15:25:06.000,2021-01-03 15:25:06.000,2021-01-03 15:25:11,2021-01-03 15:25:11,2021-01-03 15:25:11,5.0,2021-01-03 15:25:11,2.0,FINISHED,1.00,5ff1e194b6a9d73a3a9f1052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114,603cc0630a720fde100003e6,25.0,COMPLETE_NONPARTNER_RECEIPT,2021-03-01 10:22:27.000,2021-03-01 10:22:27.000,null,null,null,25.0,null,2.0,REJECTED,34.96,5fc961c3b8cfca11a077dd33
1115,603d0b710a720fde1000042a,NaN,NaN,2021-03-01 15:42:41.873,2021-03-01 15:42:41.873,null,null,null,NaN,null,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1116,603cf5290a720fde10000413,NaN,NaN,2021-03-01 14:07:37.664,2021-03-01 14:07:37.664,null,null,null,NaN,null,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1117,603ce7100a7217c72c000405,25.0,COMPLETE_NONPARTNER_RECEIPT,2021-03-01 13:07:28.000,2021-03-01 13:07:28.000,null,null,null,25.0,null,2.0,REJECTED,34.96,5fc961c3b8cfca11a077dd33


## Get Item Table From Receipts

In [27]:
receipt_item_data = receipt_data[['rewardsReceiptItemList','Id']].copy()
receipt_item_data = explode(receipt_item_data,"rewardsReceiptItemList")

keys_ = ['barcode', 'description', 'finalPrice', 'itemPrice', 'needsFetchReview', 'partnerItemId', 'quanityPurchased']

receipt_item_data1 = pd.DataFrame(columns=keys_)
for i in range(len(keys_)):
    this_column = receipt_item_data1.columns[i]
    receipt_item_data1[this_column] = receipt_item_data['rewardsReceiptItemList'].apply(lambda x: explode1(x,this_column))

receipt_item_data1 = pd.concat([receipt_item_data,receipt_item_data1],axis=1)
receipt_item_data1 = pd.DataFrame(receipt_item_data1, columns=keys_).drop_duplicates().reset_index()
receipt_item_data1

,index,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,quanityPurchased
0,0,4011,ITEM NOT FOUND,26.00,26.00,False,1,None
1,1,4011,ITEM NOT FOUND,1,1,None,1,None
2,1,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,None
3,2,None,None,None,None,False,1,None
4,3,4011,ITEM NOT FOUND,28.00,28.00,False,1,None
...,...,...,...,...,...,...,...,...
5934,999,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,None,0,None
5935,999,B07BRRLSVC,thindust summer face mask - sun protection nec...,11.99,11.99,None,1,None
5936,1015,B08BGBHHP6,spigen thin fit designed for iphone 12 mini ca...,12.59,12.59,None,0,None
5937,1015,B08DQDHR2S,spigen tempered glass screen protector [glas.t...,12.99,12.99,None,1,None


## Get Brand Table

In [40]:
brand_data = pd.read_json("brands.json", lines=True)
brand_tb = pd.DataFrame(brand_data, columns=["barcode", "category", "categoryCode", "name", "topBrand", "brandCode"]).drop_duplicates()
brand_tb

,barcode,category,categoryCode,name,topBrand,brandCode
0,511111019862,Baking,BAKING,test brand @1612366101024,0.0,NaN
1,511111519928,Beverages,BEVERAGES,Starbucks,0.0,STARBUCKS
2,511111819905,Baking,BAKING,test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176
3,511111519874,Baking,BAKING,test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051
4,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827
...,...,...,...,...,...,...
1162,511111116752,Baking,BAKING,test brand @1601644365844,NaN,NaN
1163,511111706328,Breakfast & Cereal,NaN,Dippin Dots® Cereal,NaN,DIPPIN DOTS CEREAL
1164,511111416173,Candy & Sweets,CANDY_AND_SWEETS,test brand @1598639215217,NaN,TEST BRANDCODE @1598639215217
1165,511111400608,Grocery,NaN,LIPTON TEA Leaves,0.0,LIPTON TEA Leaves


## Get cpg table

In [56]:
cpg_data = brand_data[['cpg','barcode']].copy()
cpg_data = explode(cpg_data,"cpg")

keys_ = ['$ref', '$id']

cpg_data1 = pd.DataFrame(columns=keys_)
for i in range(len(keys_)):
    this_column = cpg_data1.columns[i]
    cpg_data1[this_column] = cpg_data['cpg'].apply(lambda x: explode1(x,this_column))

cpg_data1 = pd.concat([cpg_data,cpg_data1],axis=1)
cpg_data1 = pd.DataFrame(cpg_data1, columns=keys_)
cpg_data1['$id'] = cpg_data1['$id'].apply(lambda x: list(x.values())[0]).drop_duplicates()
cpg_data1


,$ref,$id
0,Cogs,601ac114be37ce2ead437550
1,Cogs,5332f5fbe4b03c9a25efd0ba
2,Cogs,601ac142be37ce2ead437559
3,Cogs,NaN
4,Cogs,5332fa12e4b03c9a25efd1e7
...,...,...
1162,Cogs,5f77274dbe37ce6b592e90bf
1163,Cogs,NaN
1164,Cogs,NaN
1165,Cogs,NaN


## Get User Table

In [59]:
user_data = pd.read_json("users.json", lines=True)
user_data['_id'] = user_data['_id'].apply(lambda x: list(x.values())[0])
user_data = user_data.rename(columns={'_id':'Id'})
user_data['createdDate'] = user_data['createdDate'].apply(lambda x: date(x))
user_data['lastLogin'] = user_data['lastLogin'].apply(lambda x: date(x))
user_tb = pd.DataFrame(user_data, columns=["_id", "state", "createdDate", "lastLogin", "role", "active", "signUpSource"]).drop_duplicates()
user_tb 

,_id,state,createdDate,lastLogin,role,active,signUpSource
0,NaN,WI,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858000,consumer,True,Email
3,NaN,WI,2021-01-03 15:25:30.554,2021-01-03 15:25:30.597000,consumer,True,Email
6,NaN,WI,2021-01-03 15:25:28.354,2021-01-03 15:25:28.392000,consumer,True,Email
7,NaN,WI,2021-01-03 15:24:39.626,2021-01-03 15:24:39.665000,consumer,True,Email
9,NaN,WI,2021-01-03 15:25:37.564,2021-01-03 15:25:37.599000,consumer,True,Email
...,...,...,...,...,...,...,...
435,NaN,NH,2020-12-03 22:08:03.936,2021-02-26 22:39:16.799000,fetch-staff,True,Email
455,NaN,NaN,2020-11-05 15:17:09.396,2021-03-04 16:02:02.026000,fetch-staff,True,Email
456,NaN,AL,2020-11-04 22:29:33.309,2021-03-04 07:21:58.047000,fetch-staff,True,Google
462,NaN,IL,2017-07-11 15:13:11.771,2021-03-04 19:07:49.770000,fetch-staff,True,NaN


## Convert json data to csv

In [60]:
brand_tb.to_csv('brands.csv',encoding='utf-8')
cpg_data1.to_csv('cpg.csv',encoding='utf-8')
receipt_item_data1.to_csv('receiptitems.csv',encoding='utf-8')
receipt_tb.to_csv('receipts.csv',encoding='utf-8')
user_tb.to_csv('users.csv',encoding='utf-8')